# Prepare datasets for epitope prediction

In [1]:
import json
import os
from copy import deepcopy
import random

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%reload_ext autoreload
%autoreload 2

In [2]:
if not os.path.isdir('/home/yuan/results/epitope/seq_vector_1d'):
    os.mkdir('/home/yuan/results/epitope/seq_vector_1d')

outdir = '/home/yuan/results/epitope'

seq_dir = os.path.join(outdir, 'seq')
if not os.path.isdir(seq_dir):
    os.mkdir(seq_dir)


json_dir = '/home/yuan/data/omics_data/epitope/mysql'

sizes = [7, 9, 10, 12, 15, 20, 30, 50]
info = {i:0 for i in ['wrong_size',  'seq', 'err']}

## 1. epitopes

In [ ]:
# collect all epitopes
from utils import Utils

key = 'epitope'
n, m = 0, 0
pool = {}
outfile = f'/home/yuan/results/epitope/{key}.txt'
with open(outfile, 'w') as f:
    seq_iter = Utils.get_epitope_seq()
    for seq in seq_iter:
        if seq not in pool:
            f.write(seq + '\t' + key + '\n')
            pool[seq] = 1
            n += 1
        else:
            pool[seq] += 1
            m += 1
print(f"Number of unique epitopes: {n}")
print(f"Number of duplicated epitopes: {m}")

## 2. non epitope sequences

### random sequences

In [ ]:
# create random sequences by fixed size
from utils import Utils

from constants import PROPERTY
aa = list(PROPERTY)

for size in sizes:
    outfile = os.path.join(seq_dir, f'random_{size}.txt')
    print(outfile)
    with open(outfile, 'w') as f:
        for _ in range(4_000_000):
            seq = np.random.choice(aa, size)
            seq = ''.join(seq)
            f.write(seq + '\t' + 'random\n')

In [ ]:
# collect wrong sequence by random sequences
from utils import Utils

from constants import PROPERTY
aa = list(PROPERTY)

key = 'random'
n, m, pool = 0, 0, {}
outfile = os.path.join(outdir, f'{key}.txt')
with open(outfile, 'w') as f:
    seq_iter = Utils.get_epitope_seq()
    for seq in seq_iter:
        if len(seq) >= 6 and seq not in pool:
            _seq = np.random.choice(aa, len(seq))
            _seq = ''.join(_seq)
            f.write(_seq + '\t' + key + '\n')
            pool[seq] = 1
            n += 1
        else:
            m += 1
print(f"Number of random sequence: {n}-{m}")

### shuffle epitopes

In [ ]:
# collect wrong sequence by shuffling epitopes
from utils import Utils

key = 'shuffle'
n , m, pool = 0, 0, {}
outfile = os.path.join(outdir, f'{key}.txt')
with open(outfile, 'w') as f:
    seq_iter = Utils.get_epitope_seq()
    for seq in seq_iter:
        if len(seq) >= 6 and seq not in pool:
            _seq = list(seq)
            np.random.shuffle(_seq)
            _seq = ''.join(_seq)
            f.write(_seq + '\t' + key + '\n')
            pool[seq] = 1
            n += 1
        else:
            m += 1
print(f"Number of shuffled epitopes: {n}")
print(f"Number of skipped epitopes: {m}")

### non-epitopes: other sequences

In [ ]:
# build non-epitopes from antigens
from utils import Utils
from isolate_aa import IsolateAA


num, key = 2, f'other'
n, m, pool = 0, 0, {}
outfile = os.path.join(outdir, f'{key}.txt')
with open(outfile, 'w') as f:
    rec_iter = Utils.get_data()
    for acc, record in rec_iter:
        slicer = IsolateAA(record)
        try:
            seqs = slicer.random_size_seq(num=num)
            for seq in seqs:
                if seq not in pool:
                    f.write(seq + '\t' + key + '\n')
                    pool[seq] = 1
                    n += 1
                else:
                    m += 1
        except Exception as e:
            m +=1
        if n % 10_000 == 0:
            print(n, end=',')
print(f"Number of random seq: {n}. failed: {m}")

## 3. AA physical-chemical

### statistics

In [ ]:
# epitopes
from aa_comp import AAComp

info = AAComp.run(f'{outdir}/epitope.txt')

In [ ]:
# non-epitopes in antigens
from aa_comp import AAComp

info = AAComp.run(f'{outdir}/other.txt')

In [ ]:
# random sequences in antigens
from aa_comp import AAComp

info = AAComp.run(f'{outdir}/random.txt')

In [ ]:
# shuffled sequencs of epitopes in antigens
from aa_comp import AAComp

info = AAComp.run(f'{outdir}/shuffle.txt')

In [ ]:
import pandas as pd
from collections import Counter

# combine features
infile = '/home/yuan/results/epitope/epitope_combined_features.txt'
df1 = pd.read_csv(infile, sep='\t', header=0, index_col=None)
infile = '/home/yuan/results/epitope/other_combined_features.txt'
df2 = pd.read_csv(infile, sep='\t', header=0, index_col=None)
df = pd.concat([df1, df2], axis=0)
print('all features', df.shape)

is_data = df.iloc[:,2:].sum() > 0
print('filter zeros:', Counter(is_data))
ft_names = list(is_data[is_data].index)
print('none-zeros:', len(ft_names))
df = df[['seq','label']+ft_names]
print('after filtered:', df.shape)

In [ ]:
# sequence and features are columns
outfile = '/home/yuan/results/epitope/combined_features.txt'
df.to_csv(outfile, header=True, index=False, sep='\t')

In [ ]:
# # sequence, phyical-chemicals, and frequency are combined in one column
# outfile = '/home/yuan/results/epitope/combined_text.csv'
# with open(outfile, 'w') as f:
#     for i, row in df.iterrows():
#         _seq = ' '.join(list(row['seq']))
#         _ft = row[2:]
#         _ft = _ft.astype(str).str.cat(sep=' ')
#         row = f"{_seq} | {_ft},{row['label']}\n"
#         f.write(row)

In [ ]:
# # sequence, phyical-chemical are combined in one column
# outfile = '/home/yuan/results/epitope/combined_features_1.csv'
# with open(outfile, 'w') as f:
#     for i, row in df.iterrows():
#         _seq = ' '.join(list(row['seq']))
#         _ft = row[2:12]
#         _ft = _ft.astype(str).str.cat(sep=' ')
#         row = f"{_seq} | {_ft},{row['label']}\n"
#         f.write(row)

In [ ]:
list(df.iloc[:,2:12])

### represent seq by features

In [ ]:
from encode_aa import EncodeAA

EncodeAA().physical_chemical_text('ATGILLG')

In [1]:
%reload_ext autoreload
%autoreload 2
    
from aa_comp import AAComp
outdir = '/home/yuan/results/epitope'

# deprecated
# AAComp.represent_seq_1(f'{outdir}/epitope.txt')
# AAComp.represent_seq_1(f'{outdir}/other.txt')

# 
AAComp.represent_seq_2(f'{outdir}/epitope.txt')
AAComp.represent_seq_2(f'{outdir}/other.txt')

(1798795, 2)
{'A': 'C', 'R': 'M B B B', 'N': 'N J B', 'D': 'E B', 'C': 'K B', 'Q': 'N J B B', 'E': 'E B B', 'G': '', 'H': 'H B', 'I': 'C B I', 'L': 'D A B', 'K': 'N B B B B', 'M': 'C S B B', 'F': 'F B', 'P': 'P', 'S': 'S', 'T': 'C T', 'W': 'F W B', 'Y': 'O F B', 'V': 'D A'}
(1798581, 3)
                     seq                                               text  \
0          RPIAEYLNTQKDM   B B B P C B I C E B B O F B D A B N J B C T N...   
1              ARFDSVFGK                M B B B F B E B S D A F B N B B B B   
2             ARFDSVFGKF            M B B B F B E B S D A F B N B B B B F B   
3            ARFDSVFGKFL      M B B B F B E B S D A F B N B B B B F B D A B   
4               DAFVAYHI                      B C F B D A C O F B H B C B I   
...                  ...                                                ...   
1798790         WADNEPNN                W B C E B N J B E B B P N J B N J B   
1798791  TGALFKHSKKGPRAS   T C D A B F B N B B B B H B S N B B B B N B B...   
1

In [7]:
import pandas as pd
from encode_aa import EncodeAA

# read df: default two columns
df = pd.read_csv(f'{outdir}/epitope.txt', sep='\t', header=None, index_col=None)
df.columns = ['seq', 'label']
print(df.shape)

encoder = EncodeAA()
# phy_che = df['seq'].map(encoder.physical_chemical_text)
smiles = df['seq'].map(encoder.aa_smiles)
dfv= pd.DataFrame({
    'seq': df['seq'],
    'text': smiles,
    'label': df['label'],
})
print(dfv.shape)

(1798795, 2)
{'A': 'C', 'R': 'MBBB', 'N': 'NJC', 'D': 'RB', 'C': 'GB', 'Q': 'NJBB', 'E': 'RBB', 'G': 'H', 'H': 'ZB', 'I': 'CBE', 'L': 'DAB', 'K': 'NBBBB', 'M': 'CSBB', 'F': 'XB', 'P': 'P', 'S': 'Q', 'T': 'CT', 'W': 'XYBB', 'Y': 'OXB', 'V': 'DA'}
(1798795, 3)


In [8]:
dfv[dfv['text'].isna()]

,seq,text,label
9188,XREGGVLRVQPRATRFTFRTARQVPRLGVML,None,epitope
77506,XAAAMPLGLPLRLLVLLLVG,None,epitope
89778,XLGLPRVLA,None,epitope
276813,AXVLVNAIVFKGLWE,None,epitope
350451,AWGHITISTAAXYRNAVVEQ,None,epitope
...,...,...,...
855400,QXRAPRITFGGPSDST,None,epitope
855442,SDNGPQXRAPRITFGG,None,epitope
855529,XRAPRITFGGPSDSTD,None,epitope
1407812,GSPNPIVLPXPPPPP,None,epitope


In [6]:
dfv.dropna().shape

(1383957, 3)

In [7]:
dfv[dfv['text'].str.len()<2]

,seq,text,label
9213,XXXXXXXXXXXXXXX,,other
9393,AXXXXXXXXXXXXXX,,other
883811,XXXXXXXXXXXXX,,other


In [4]:
a= 'ATB'
set(a).difference([3,1])

{'A', 'B', 'T'}

## retrieve sequence segment

In [ ]:
# slice epitopes
def kmer_expand(size):
    outfile = os.path.join(seq_dir, f'epitope_{size}_kmer_expand.txt')
    print(outfile)
    with open(outfile, 'w') as f:
        rec_iter = Utils.scan_json_record(json_dir)
        for acc, record in rec_iter:
            slicer = IsolateAA(record)
            try:
                epi_seq = slicer.slice_kmer_expand(size)
                for seq in epi_seq:
                    if len(seq) == size:
                        f.write(seq + '\t' + 'epitope\n')
                        info['seq'] += 1
                    else:
                        info['wrong_size'] += 1
            except Exception as e:
                info['err'] += 1
    print(f"Statistics of {size}: {info}")

for size in sizes:
    kmer_expand(size)

In [ ]:
def shrink_expand(size):
    outfile = os.path.join(seq_dir, f'epitope_{size}_shrink_expand.txt')
    print(outfile)
    with open(outfile, 'w') as f:
        rec_iter = Utils.scan_json_record(json_dir)
        for acc, record in rec_iter:
            slicer = IsolateAA(record)
            try:
                epi_seq = slicer.slice_shrink_expand(size)
                for seq in epi_seq:
                    if len(seq) == size:
                        f.write(seq + '\t' + 'epitope\n')
                        info['seq'] += 1
                    else:
                        info['wrong_size'] += 1
            except Exception as e:
                info['err'] += 1
    print(f"Statistics of {size}: {info}")

for size in sizes:
    shrink_expand(size)

In [ ]:
# random other sequences
def random_other(size):
    outfile = os.path.join(seq_dir, f'other_{size}.txt')
    print(outfile)
    with open(outfile, 'w') as f:
        rec_iter = Utils.scan_json_record(json_dir)
        for acc, record in rec_iter:
            slicer = IsolateAA(record)
            num_epi = slicer.num_epitopes()
            try:
                other_seq = slicer.random_other_seq(size, num_epi)
                for seq in other_seq:
                    if len(seq) == size:
                        f.write(seq + '\t' + 'other\n')
                        info['seq'] += 1
                    else:
                        info['wrong_size'] += 1
            except Exception as e:
                info['err'] += 1
    print(f"Statistics of {size}: {info}")

for size in sizes:
    random_other(size)

## encode

In [ ]:
# encode 
encoder = EncodeAA()

outdir = '/home/yuan/results/epitope/seq_vector_1d'
indir = '/home/yuan/results/epitope/seq'
df_iter = Utils.scan_text(indir, '\t')
for df, file_name in df_iter:
    df = df.dropna()
    dfv = df.apply(lambda x: encoder.vector_1d(x[0], x[1]), axis=1, result_type='expand')
    outfile = os.path.join(outdir, file_name)
    dfv.to_csv(outfile, header=False, index=False, sep='\t')
    print(outfile)